In [108]:
import pandas as pd
import os
from utils import *

In [109]:
from dotenv import load_dotenv
load_dotenv()


True

In [110]:
token = os.getenv('data_access_token')
client_id = int(os.getenv('client_id'))
business_timezone_offset = int(os.getenv('business_timezone_offset'))
business_name = os.getenv("business_name")



In [111]:
#read all data
orders_header = pd.read_csv(f'../../data/{client_id}/raw/order_header.csv')
orders_details = pd.read_csv(f'../../data/{client_id}/raw/order_details.csv')
products = pd.read_csv(f'../../data/{client_id}/raw/products.csv')
branches = pd.read_csv(f'../../data/{client_id}/raw/branches.csv')
categories = pd.read_csv(f'../../data/{client_id}/raw/categories.csv')






# Orders Headers

In [112]:
orders_header.head()

,id,client_id,branch_id,source,type,status,total_price,created_at,updated_at,ordered_at
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,15.00,2022-10-02 11:03:36,2022-11-28 09:26:24,2022-10-02 14:03:36
1,5babceee-5385-4fa5-b25f-4f1638a70696,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,6.80,2022-10-02 10:41:05,2022-11-21 07:53:39,2022-10-02 13:41:05
2,3c811baa-93bc-4200-9204-fdee8932f020,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,5.00,2022-10-02 10:39:04,2022-10-02 10:47:48,2022-10-02 13:39:04
3,7e7a67f8-849d-42da-9cac-e29d3144e9ec,1,975b3d24-cb71-4df8-930e-054bcd67f90c,1,2,4,20.75,2022-10-02 13:36:54,2022-11-28 11:43:54,2022-10-02 16:36:54
4,e02a78da-8a3e-4dad-8a27-ecc739f8d8a4,1,975b3d24-cb71-4df8-930e-054bcd67f90c,1,1,4,30.00,2022-10-02 13:15:22,2022-11-28 11:44:07,2022-10-02 16:15:22


In [113]:
orders_types = {1:'Dine In', 2:'Pick Up', 3:'Delivery', '4':'Drive Thru'}
orders_sources = {1:'Cashier', 2:'API', 3:'Call Center'}
orders_statuses = {1:'Pending', 2:'Active', 3:'Declined', 4:'Closed', 5:'Returned', 6:'Joined', 7:'Void'}

#---------------

orders_header['type'] = orders_header['type'].map(orders_types)
orders_header['source'] = orders_header['source'].map(orders_sources)
orders_header['status'] = orders_header['status'].map(orders_statuses)


In [114]:
orders_header['ordered_at'] = pd.to_datetime(orders_header['ordered_at'])


In [115]:
orders_header.drop_duplicates(inplace=True)

In [116]:
orders_header.isnull().sum()

id             0
client_id      0
branch_id      0
source         0
type           0
status         4
total_price    0
created_at     0
updated_at     0
ordered_at     0
dtype: int64

In [117]:
orders_header.head()

,id,client_id,branch_id,source,type,status,total_price,created_at,updated_at,ordered_at
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,1,975b3d24-ce3d-4801-9c11-582a817cc591,Cashier,Dine In,Closed,15.00,2022-10-02 11:03:36,2022-11-28 09:26:24,2022-10-02 14:03:36
1,5babceee-5385-4fa5-b25f-4f1638a70696,1,975b3d24-ce3d-4801-9c11-582a817cc591,Cashier,Dine In,Closed,6.80,2022-10-02 10:41:05,2022-11-21 07:53:39,2022-10-02 13:41:05
2,3c811baa-93bc-4200-9204-fdee8932f020,1,975b3d24-ce3d-4801-9c11-582a817cc591,Cashier,Dine In,Closed,5.00,2022-10-02 10:39:04,2022-10-02 10:47:48,2022-10-02 13:39:04
3,7e7a67f8-849d-42da-9cac-e29d3144e9ec,1,975b3d24-cb71-4df8-930e-054bcd67f90c,Cashier,Pick Up,Closed,20.75,2022-10-02 13:36:54,2022-11-28 11:43:54,2022-10-02 16:36:54
4,e02a78da-8a3e-4dad-8a27-ecc739f8d8a4,1,975b3d24-cb71-4df8-930e-054bcd67f90c,Cashier,Dine In,Closed,30.00,2022-10-02 13:15:22,2022-11-28 11:44:07,2022-10-02 16:15:22


In [118]:
orders_header['status'].fillna('Void', inplace=True)

# Order Details

In [119]:
orders_details.head()


,id,header_id,product_id,category_id,client_id,quantity,price,created_at,updated_at
0,bb547c16-a622-43c2-8836-a21c621c3279,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,4.5,2022-10-02 11:03:36,2022-11-28 09:26:24
1,2a6c2988-acee-4fb4-a539-03a01575be11,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-96ee-41d6-9ed2-c2fb747e3f0a,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,5.5,2022-10-02 11:03:36,2022-11-28 09:26:24
2,3d9a991f-4358-4fab-9fca-85d565931cdc,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-92bd-46f1-86c1-183d3a1c3c32,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,5.0,2022-10-02 11:03:36,2022-11-28 09:26:24
3,b3cec149-3f7e-410e-908a-93252cb02725,5babceee-5385-4fa5-b25f-4f1638a70696,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,4.5,2022-10-02 10:41:05,2022-11-21 07:53:39
4,d4652f41-b515-4b62-b56e-a90fb5faa748,5babceee-5385-4fa5-b25f-4f1638a70696,975b583f-2d41-4563-82d1-b09078a2e80a,975b5737-cf8c-4ef3-a445-7c4a71f349c5,1,1,0.3,2022-10-02 10:41:05,2022-11-21 07:53:39


# Products

In [120]:
products.head()

,id,client_id,sku,slug,name,category_id,is_active,is_stock,price,created_at,updated_at,deleted_at
0,975b583e-8e4e-4500-b416-e77dd352726a,1,psk-14,تشكس-فرايز-زنجر,تشكس فرايز زنجر,975b5737-c982-4e3f-9faf-083ca9dc56ac,True,False,5.00,2022-09-26 09:49:43,2022-10-02 10:36:36,NaN
1,975b583e-90a9-420a-804e-b41dddbe0971,1,psk-5,كلاسيك,كلاسيك,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,True,False,1.50,2022-09-26 09:49:43,2022-10-02 10:36:36,NaN
2,975b583e-90af-4319-87d3-022d8f1936d4,1,psk-13,تشكس-ستريبس,تشكس ستريبس,975b5737-ccc4-4c1f-8599-5e246eee4a3b,True,False,4.50,2022-09-26 09:49:43,2022-10-02 10:36:36,NaN
3,975b583e-9145-4e05-90e7-9fa596c38888,1,psk-16,فرايز-باجبنة,فرايز باجبنة,975b5737-c982-4e3f-9faf-083ca9dc56ac,True,False,2.50,2022-09-26 09:49:43,2022-10-02 10:36:36,NaN
4,975b583e-91e6-424e-851d-4de6a37813f5,1,psk-9,اربيان-دجاج-l,اربيان دجاج l,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,True,False,3.75,2022-09-26 09:49:43,2022-12-29 08:58:43,NaN


In [121]:

if business_name == 'Shawarma 4 Chicks':
    map_product_translation = {
        'تشكس فرايز زنجر': 'Chicks Fries Zinger',
        'كلاسيك': 'Classic',
        'تشكس ستريبس': 'Chicks Strips',
        'فرايز باجبنة': 'Fries with Cheese',
        'اربيان دجاج l': 'Arabian Chicken L',
        'تشكس برجر': 'Chicks Burger',
        'تشيزي': 'Cheesy',
        'اوريجنال': 'Original',
        'بيج ميل': 'Big Meal',
        'فرايز': 'Fries',
        'ديبي برجر': 'Debbie Burger',
        'تشكس فرايز شاورما': 'Chicks Fries Shawarma',
        'موهيتو': 'Mojito',
        'كوكتيل': 'Cocktail',
        'برو': 'Brew',
        'هوت دجاج': 'Hot Chicken',
        'ذا بارتي': 'The Party',
        'بيبسي': 'Pepsi',
        'اربيان دجاج xl': 'Arabian Chicken XL',
        'شيك دجاج': 'Chicken Shake',
        'مياه': 'Water',
        'سفن': '7UP',
        'هوت صوص': 'Hot Sauce',
        'مرندا': 'Mirinda',
        'بيبسي دايت': 'Diet Pepsi',
        'ثوم': 'Garlic',
        'سفن دايت': 'Diet 7UP',
        'مخلل': 'Pickles',
        'جبنة': 'Cheese',
        'تشكس صوص': 'Chicks Sauce',
        'رانش': 'Ranch',
        'دبس رمان': 'Pomegranate Molasses',
        'راب شيكس': 'Wrap Chicks',
        'صوص ثوم حار': 'Spicy Garlic Sauce',
        'فرشلي': 'Freshly',
        '5 اوريجنال عرض كريم 5JD': '5 Original Careem Offer 5JD',
        'عرض بيج شيكس / عائلية  / 10JOD': 'Big Chicks Offer / Family / 10JOD',
        'عرض 2 وجبة لاج دجاج 5 د': '2 Large Chicken Meal Offer 5 JD',
        'وجبة مشاركة شاورما +زنجر + 2 بيبسي // كريم+ طلبات': 'Sharing Meal Shawarma + Zinger + 2 Pepsi // Careem + Orders',
        'وجبة مشاركة برغر + زنجر + 2 بيبسي // كريم+ طلبات': 'Sharing Meal Burger + Zinger + 2 Pepsi // Careem + Orders',
        'وجبة موظفين': 'Staff Meal',
        'عصير موسمي': 'Seasonal Juice',
        'وجبة لارج': 'Large Meal',
        'بيج شيكس عرض فوديز 9 د': 'Big Chicks Fries Offer 9 JD',
        'وجبة مشاركة برجر عرض 10 د': 'Sharing Meal Burger Offer 10 JD',
        'عرض وجبة عائلية 10د': 'Family Meal Offer 10 JD',
        'عرض وجبة الاصدقاء  / مشاركة شاورما 6.500 JOD': 'Friends Meal Offer / Sharing Shawarma 6.500 JOD',
        'عرض كريم 2 وجبة لارج 5 د': 'Careem Offer 2 Large Meals 5 JD',
        'عرض كريم XL اربيان 50%': 'Careem Offer XL Arabian 50%',
        'عرض كريم 2 اورجينال+1 بطاط +1 بيبسي': 'Careem Offer 2 Original + 1 Fries + 1 Pepsi',
        'شيكس برغر / طلبات /  بيبسي +بطاطا': 'Chicks Burger / Orders / Pepsi + Fries',
        'دبي برغر / طلبات /  بيبسي +بطاطا': 'Debbie Burger / Orders / Pepsi + Fries',
        'وجبة مشاركة شاورما + 2 بيبسي  // 8JD': 'Sharing Meal Shawarma + 2 Pepsi // 8JD',
        'وجبة مشاركة برغر  + 2 بيبسي  // 10JD': 'Sharing Meal Burger + 2 Pepsi // 10JD',
        'شاورما بيج 12 د / عرض كريم': 'Shawarma Big 12 JD / Careem Offer'
    }
    products.name = products.name.map(map_product_translation)

In [122]:
products.name.unique().tolist()

['Chicks Fries Zinger',
 'Classic',
 'Chicks Strips',
 'Fries with Cheese',
 'Arabian Chicken L',
 'Chicks Burger',
 'Cheesy',
 'Original',
 'Big Meal',
 'Fries',
 'Debbie Burger',
 'Chicks Fries Shawarma',
 'Mojito',
 'Cocktail',
 'Brew',
 'Hot Chicken',
 'The Party',
 'Pepsi',
 'Arabian Chicken XL',
 'Chicken Shake',
 'Water',
 '7UP',
 'Hot Sauce',
 'Mirinda',
 'Diet Pepsi',
 'Garlic',
 'Diet 7UP',
 'Pickles',
 'Cheese',
 'Chicks Sauce',
 'Ranch',
 'Pomegranate Molasses',
 'Wrap Chicks',
 'Spicy Garlic Sauce',
 'Freshly',
 '5 Original Careem Offer 5JD',
 'Big Chicks Offer / Family / 10JOD',
 '2 Large Chicken Meal Offer 5 JD',
 'Sharing Meal Shawarma + Zinger + 2 Pepsi // Careem + Orders',
 'Sharing Meal Burger + Zinger + 2 Pepsi // Careem + Orders',
 'Staff Meal',
 'Seasonal Juice',
 'Large Meal',
 'Big Chicks Fries Offer 9 JD',
 'Sharing Meal Burger Offer 10 JD',
 'Family Meal Offer 10 JD',
 'Friends Meal Offer / Sharing Shawarma 6.500 JOD',
 'Careem Offer 2 Large Meals 5 JD',
 'Car

In [123]:
products['slug'] = products.name.apply(lambda x: generate_slug(x))


In [124]:
products.head()

,id,client_id,sku,slug,name,category_id,is_active,is_stock,price,created_at,updated_at,deleted_at
0,975b583e-8e4e-4500-b416-e77dd352726a,1,psk-14,chicks-fries-zinger,Chicks Fries Zinger,975b5737-c982-4e3f-9faf-083ca9dc56ac,True,False,5.00,2022-09-26 09:49:43,2022-10-02 10:36:36,NaN
1,975b583e-90a9-420a-804e-b41dddbe0971,1,psk-5,classic,Classic,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,True,False,1.50,2022-09-26 09:49:43,2022-10-02 10:36:36,NaN
2,975b583e-90af-4319-87d3-022d8f1936d4,1,psk-13,chicks-strips,Chicks Strips,975b5737-ccc4-4c1f-8599-5e246eee4a3b,True,False,4.50,2022-09-26 09:49:43,2022-10-02 10:36:36,NaN
3,975b583e-9145-4e05-90e7-9fa596c38888,1,psk-16,fries-with-cheese,Fries with Cheese,975b5737-c982-4e3f-9faf-083ca9dc56ac,True,False,2.50,2022-09-26 09:49:43,2022-10-02 10:36:36,NaN
4,975b583e-91e6-424e-851d-4de6a37813f5,1,psk-9,arabian-chicken-l,Arabian Chicken L,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,True,False,3.75,2022-09-26 09:49:43,2022-12-29 08:58:43,NaN


In [125]:
products.sku.unique().tolist()

['psk-14',
 'psk-5',
 'psk-13',
 'psk-16',
 'psk-9',
 'psk-11',
 'psk-6',
 'psk-4',
 'psk-8',
 'psk-17',
 'psk-12',
 'psk-15',
 'psk-20',
 'psk-18',
 'psk-3',
 'psk-2',
 'psk-19',
 'psk-21',
 'psk-10',
 'psk-7',
 'psk-26',
 'psk-23',
 'psk-30',
 'psk-25',
 'psk-22',
 'psk-29',
 'psk-24',
 'psk-28',
 'psk-32',
 'psk-27',
 'psk-31',
 'psk-33',
 'sk-0106',
 'sk-0107',
 'sk-0108',
 'sk-0109',
 'sk-0110',
 'sk-0111',
 'sk-0112',
 'sk-0113',
 'sk-0114',
 'sk-0115',
 'sk-0116',
 'sk-0117',
 'sk-0118',
 'sk-0119',
 'sk-0120',
 'sk-0121',
 'sk-0122',
 'sk-0123',
 'sk-0124',
 'sk-0125',
 'sk-0126',
 'sk-0128',
 'sk-0129',
 'sk-0130']

In [126]:
products[products.name.isnull()]

,id,client_id,sku,slug,name,category_id,is_active,is_stock,price,created_at,updated_at,deleted_at


In [127]:
products.sku.nunique()

56

# Branches

In [128]:
branches

,id,client_id,name,slug,opening_from,opening_to,created_at,updated_at,deleted_at
0,975b3d24-cb71-4df8-930e-054bcd67f90c,1,4Chicks Abdoun,4chicks-abdoun,09:00,03:00,2022-09-26 08:33:56,2022-10-02 08:09:02,NaN
1,975b3d24-ce3d-4801-9c11-582a817cc591,1,4Chicks 7th circle,4chicks-7th-circle,10:00,03:00,2022-09-26 08:33:56,2022-10-02 08:09:24,NaN
2,976744f0-20ac-4dd5-a06a-6a1ee9ffa7b5,1,4Chicks Al-Jubeha,4chicks-al-jubeha,10:00,03:00,2022-10-02 08:05:40,2022-10-02 08:22:37,NaN


In [129]:
branches['opening_from'] = pd.to_datetime(branches['opening_from'])
branches['opening_to'] = pd.to_datetime(branches['opening_to'])

# branches['opening_from'] = branches['opening_from'].dt.hour
# branches['opening_to'] = branches['opening_to'].dt.hour

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11056\3052712850.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  branches['opening_from'] = pd.to_datetime(branches['opening_from'])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_11056\3052712850.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  branches['opening_to'] = pd.to_datetime(branches['opening_to'])


In [130]:
branches

,id,client_id,name,slug,opening_from,opening_to,created_at,updated_at,deleted_at
0,975b3d24-cb71-4df8-930e-054bcd67f90c,1,4Chicks Abdoun,4chicks-abdoun,2023-06-12 09:00:00,2023-06-12 03:00:00,2022-09-26 08:33:56,2022-10-02 08:09:02,NaN
1,975b3d24-ce3d-4801-9c11-582a817cc591,1,4Chicks 7th circle,4chicks-7th-circle,2023-06-12 10:00:00,2023-06-12 03:00:00,2022-09-26 08:33:56,2022-10-02 08:09:24,NaN
2,976744f0-20ac-4dd5-a06a-6a1ee9ffa7b5,1,4Chicks Al-Jubeha,4chicks-al-jubeha,2023-06-12 10:00:00,2023-06-12 03:00:00,2022-10-02 08:05:40,2022-10-02 08:22:37,NaN


# Categories

In [131]:
categories

,id,client_id,name,slug,created_at,updated_at,deleted_at
0,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,1,Shawarma,شاورما,2022-09-26 09:46:50,2022-09-26 09:46:50,NaN
1,975b5737-c982-4e3f-9faf-083ca9dc56ac,1,Fries,فرايز,2022-09-26 09:46:50,2022-09-26 09:46:50,NaN
2,975b5737-cad1-4031-9319-5e3a7addba42,1,Drinks,مشروبات,2022-09-26 09:46:50,2022-09-26 09:46:50,NaN
3,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,Chicken burger,برغر-دجاج,2022-09-26 09:46:50,2022-09-26 09:46:50,NaN
4,975b5737-cf8c-4ef3-a445-7c4a71f349c5,1,Sauces,صوصات,2022-09-26 09:46:50,2022-09-26 09:46:50,NaN
5,975b5737-d227-4b7c-9962-3a7b09103597,1,Juices,عصائر,2022-09-26 09:46:50,2022-09-26 09:46:50,NaN


# To Be Removed

In [132]:
from datetime import datetime
products['deleted_at'] = datetime.now()
branches['deleted_at'] = datetime.now()
categories['deleted_at'] = datetime.now()


# Write

In [133]:
orders_header.to_csv(f'../../data/{client_id}/processed/order_header.csv', index=False)

In [134]:
orders_details.to_csv(f'../../data/{client_id}/processed/order_details.csv', index=False)

In [135]:
products.to_csv(f'../../data/{client_id}/processed/products.csv', index=False)


In [136]:
branches.to_csv(f'../../data/{client_id}/processed/branches.csv', index=False)

In [137]:
categories.to_csv(f'../../data/{client_id}/processed/categories.csv', index=False)